In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [2]:
%%px
# Import the libraries

import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import math
import sys
import os
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

In [3]:
%%px

Nx = isotropic1024coarse['nx']
Ny = isotropic1024coarse['ny']
Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']
Ly = isotropic1024coarse['ly']
Lz = isotropic1024coarse['lz']
nu = isotropic1024coarse['nu']

# Computational Domain


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))
    
nx=Nx//nproc
ny=Ny
nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0#1.024# 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:0] 
n_proc = 8
rank = 0


In [4]:
%%px

vx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = True

folder = "/home/idies/workspace/scratch"
filename = "cache-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(load_from_file):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    vx = content['vx']
    vy = content['vy']
    vz = content['vz']
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    lJHTDB = libJHTDB()
    lJHTDB.initialize()
    ud = []
    for k in range(slabs):
        if(rank==0):
            print(k)
        start = np.array([rank*nx+k*chkSz, 0, 0],dtype=np.int)
        width = np.array([chkSz,ny,nz],dtype=np.int)
        #start = np.array([ 0, 0, rank*nx+k*chkSz],dtype=np.int)
        ud.append(lJHTDB.getRawData(time,start,width, 
                                    data_set = 'isotropic1024coarse',
                                    getFunction = 'Velocity') )
    
    lJHTDB.finalize()
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    u = np.concatenate(ud,axis=2)
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Concatenate: {0:.2f} seconds\n'.format(t1-t2))
    
    rsh = np.transpose(u,(2,1,0,3))
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Transpose: {0:.2f} seconds\n'.format(t2-t1))
    ##########################################
    
    vx[:,:,:] = rsh[:,:,:,0]
    vy[:,:,:] = rsh[:,:,:,1]
    vz[:,:,:] = rsh[:,:,:,2]
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Splitting: {0:.2f} seconds\n'.format(t2-t1))
    
    np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))


if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:0] 
Starting the loading process
Finished loading
Load from disk: 10.83 seconds
vx shape = (128, 1024, 1024)


In [5]:
%%px
comm.Barrier(); t1=MPI.Wtime()
# Get wavenumber:
myEnergySpc=EnergySpectrum()
kx,ky,kz=myEnergySpc.FindWavenumber(nx,ny,nz,rank)
k2=np.zeros((nx,ny,nz_half+1), dtype='float32')
np.copyto(k2,kx*kx+ky*ky+kz*kz)
k2[0,0,0]=1e-6

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate all wavenumbers: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate all wavenumbers: 10.37 seconds


In [6]:
%%px 

fft = FFT3Dfield_new(nx,ny,nz,nproc,rank)

In [7]:
%%px

cvx = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvy = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvz = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')

In [8]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cvx[:,:,:] = fft.forward3Dfft(vx,nx,ny,nz,nproc,rank)
cvy[:,:,:] = fft.forward3Dfft(vy,nx,ny,nz,nproc,rank)
cvz[:,:,:] = fft.forward3Dfft(vz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial FFT: 81.75 seconds


In [9]:
%%px
comm.Barrier(); t1=MPI.Wtime()

bvx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
bvy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
bvz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial FFT: 5.70 seconds


In [10]:
%%px
comm.Barrier(); t1=MPI.Wtime()

bvx[:,:,:]=fft.backward3Dfft(cvx,nx,ny,nz,nproc,rank)
bvy[:,:,:]=fft.backward3Dfft(cvy,nx,ny,nz,nproc,rank)
bvz[:,:,:]=fft.backward3Dfft(cvz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial iFFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial iFFT: 71.19 seconds


In [11]:
%%px

e = 0.5*np.average(vx**2+vy**2+vz**2)
be = 0.5*np.average(bvx**2+bvy**2+bvz**2)

In [12]:
%%px

print("e = "+str(e)+", be = "+str(be))

[stdout:0] e = 0.735254764557, be = 0.735254526138
[stdout:1] e = 0.643365740776, be = 0.643365621567
[stdout:2] e = 0.631311774254, be = 0.63131159544
[stdout:3] e = 0.739439487457, be = 0.739439308643
[stdout:4] e = 0.779128789902, be = 0.779128670692
[stdout:5] e = 0.548337519169, be = 0.54833739996
[stdout:6] e = 0.662315309048, be = 0.662315249443
[stdout:7] e = 0.722599923611, be = 0.722599804401


In [13]:
%%px

dl2norm = np.average((vx-bvx)**2 + (vy-bvy)**2 + (vz-bvz)**2)

In [18]:
%%px

print("L2 average of u minus fourier back :" + str(np.sqrt(dl2norm)))

KeyboardInterrupt: 

In [16]:
%%px

avge=np.zeros(1,dtype='float32')

comm.Reduce([e,MPI.REAL],[avge,MPI.REAL],op=MPI.SUM)
avge = avge[0]/nproc
avge = comm.bcast(avge, root=0)

KeyboardInterrupt: 

In [17]:
%%px

print(e)

KeyboardInterrupt: 